This is the Saildrone and GOES 16 SST collocation code. 
It is made to run either on your local computer or on pythonanywhere cloud service.
It uses opendap calls to get the data, so it is easier to run on pythonanywhere.
The only input file it uses is the single netcdf file of all the Saildrone data.

In [1]:
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from netCDF4 import Dataset, date2index, num2date
from palettable import colorbrewer
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
def listFD(url, ext=''):
    page = requests.get(url).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if (node.get('href').endswith(ext) and node.get('href').startswith('2'))]
import datetime as dt
from math import atan2
import xarray as xr



In [2]:
#filename_goes_sst = 'F:/data/cruise_data/saildrone/baja-2018/goes_sst_collocation_data.npy'
#filename_usv='F:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1.nc'
filename_goes_sst = './goes_sst_collocation_data.nc'
#filename_usv='./saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1.nc'
filename_usv='f:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1_withloggers.nc'
ds_usv = xr.open_dataset(filename_usv)


In [6]:
ds_usv.time[0,1].dt.dayofyear

<xarray.DataArray 'dayofyear' ()>
array(101, dtype=int64)
Coordinates:
    trajectory  float32 1002.0
    time        datetime64[ns] ...
    latitude    float64 ...
    longitude   float64 ...

In [30]:
ds_usv.time[0,1].dt.dayofyear
ds_usv.time[0,1]+ np.timedelta64(days=-1) 

<xarray.DataArray 'time' ()>
array('2018-04-11T18:01:00.000000000', dtype='datetime64[ns]')
Coordinates:
    trajectory  float32 1002.0
    time        datetime64[ns] ...
    latitude    float64 ...
    longitude   float64 ...

In [31]:
ilen=(len(ds_usv.latitude['obs']))
lats_usv=ds_usv.latitude[0,:].values
lons_usv=ds_usv.longitude[0,:].values
skin=ds_usv.TEMP_IR_UNCOR_MEAN[0,:].values
tair=ds_usv.TEMP_AIR_MEAN[0,:].values
bulk1=ds_usv.TEMP_CTD_MEAN[0,:].values
bulk2=ds_usv.TEMP_O2_MEAN[0,:].values
uspd=ds_usv.UWND_MEAN[0,:].values
vspd=ds_usv.VWND_MEAN[0,:].values
wing=ds_usv.HDG_WING[0,:].values
wing_ang=ds_usv.WING_ANGLE[0,:].values
pitch=ds_usv.PITCH[0,:].values
heading=ds_usv.HDG_WING[0,:].values
yaw_heading=ds_usv.HDG[0,:].values
roll=ds_usv.ROLL[0,:].values
tdim=len(skin)
wdir=np.zeros(tdim)
for i in range(0,tdim):
    wdir[i]=atan2(vspd[i],uspd[i])*180/3.14159
wspd=(uspd**2+vspd**2)**.5

In [64]:
#get lat/lon from random file, it doesn't change
file_goes='https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3C/AMERICAS/GOES16/OSISAF/v1/2018/160/20180609070000-OSISAF-L3C_GHRSST-SSTsubskin-GOES16-ssteqc_goes16_20180609_070000-v02.0-fv01.0.nc'
ds_sat = xr.open_dataset(file_goes)
lats_goes=ds_sat.lat
lons_goes=ds_sat.lon
ds_sat.close()
#initialize variables
latli_sv=-99
lonli_sv=-99
ihr_sv=-99
col_count=0  #initialize
gsst=np.zeros(tdim)
gsst_num=np.zeros(tdim)
#get goes data collocated with usv positions and times
for i in range(0,10): #tdim):
    latli = np.argmin( np.abs( lats_goes - lats_usv[i] ) )
    lonli = np.argmin( np.abs( lons_goes - lons_usv[i] ) )
    ihr = ds_usv.time[0,i].dt.hour.data
    if (latli!=latli_sv) or (lonli_sv!=lonli) or (ihr_sv!=ihr):  #need to read in new data
        syr=str(ds_usv.time[0,i].dt.year.data).zfill(4)
        smon=str(ds_usv.time[0,i].dt.month.data).zfill(2)
        sdym=str(ds_usv.time[0,i].dt.day.data).zfill(2)
        shr=str(ds_usv.time[0,i].dt.hour.data).zfill(2)
        sday_of_year=str(ds_usv.time[0,i].dt.dayofyear.data).zfill(3)
        if ds_usv.time[0,i].dt.hour.data==0:
            tem_time=ds_usv.time[0,i]+ np.timedelta64(days=-1) #increment day by -1 if 0 hr
            sday_of_year=str(tem_time.dt.dayofyear.data).zfill(3)
        adir_goes='https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3C/AMERICAS/GOES16/OSISAF/v1/'
        fgoes='0000-OSISAF-L3C_GHRSST-SSTsubskin-GOES16-ssteqc_goes16_'
        dstr=syr+smon+sdym+shr
        dstr2=syr+smon+sdym+'_'+shr
        file_goes=adir_goes+syr+'/'+sday_of_year+'/'+ dstr + fgoes +dstr2+'0000-v02.0-fv01.0.nc'
        r = requests.get(file_goes)
        if r.status_code != requests.codes.ok:
            continue    
        col_count += 1
        nc = Dataset(file_goes)
        sat_sst = nc.variables['sea_surface_temperature'][ : , latli , lonli ] 
        if sat_sst.mask:
            sat_sst=np.nan
        gsst[i] = sat_sst
        gsst_num[i] = col_count  #this is to keep track of when a new file read in or new collocation point
        nc.close()
        latli_sv=latli
        lonli_sv=lonli
        ihr_sv=ds_usv.time[0,i].dt.hour.data
        print(i,tdim,lonli.data, latli.data,ihr_sv,gsst[i])
    else:  #collocation is to same gsst point
        gsst[i] = sat_sst 
        gsst_num[i] = col_count  #this is to keep track of when a new file read in or new collocation point


0 86839 252 1956 18 nan
1 86839 251 1956 18 nan


In [111]:
#put into xarray dataset similiar in format to usv data
gsst2=np.zeros((1,tdim))
gsst_tem=np.zeros((1,tdim))
gsst2[0,:]=gsst
data_goes = xr.DataArray(gsst2, coords={'trajectory': ds_usv.trajectory,'obs': ds_usv.obs}, dims=('trajectory', 'obs'))
gsst_tem[0,:]=gsst_num
num_goes = xr.DataArray(gsst_tem, coords={'trajectory': ds_usv.trajectory,'obs': ds_usv.obs}, dims=('trajectory', 'obs'))
xr_gsst = xr.Dataset({'goes_sst': data_goes, 'collocation_index': num_goes}, 
                     coords={'trajectory':ds_usv.trajectory, 'time':ds_usv.time, 'latitude':ds_usv.latitude, 'longitude':ds_usv.longitude})   


<xarray.DataArray (trajectory: 1, obs: 86839)>
array([[nan, nan, nan, ...,  0.,  0.,  0.]])
Coordinates:
  * trajectory  (trajectory) float32 1002.0
  * obs         (obs) int64 0 1 2 3 4 5 ... 86833 86834 86835 86836 86837 86838

In [112]:
xr_gsst



<xarray.Dataset>
Dimensions:            (obs: 86839, trajectory: 1)
Coordinates:
  * obs                (obs) int64 0 1 2 3 4 5 ... 86834 86835 86836 86837 86838
  * trajectory         (trajectory) float32 1002.0
    latitude           (trajectory, obs) float64 37.81 37.81 ... 37.78 37.78
    longitude          (trajectory, obs) float64 -122.4 -122.4 ... -122.3 -122.3
    time               (trajectory, obs) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:57:00
Data variables:
    goes_sst           (trajectory, obs) float64 nan nan nan nan ... 0.0 0.0 0.0
    collocation_index  (trajectory, obs) float64 1.0 2.0 2.0 2.0 ... 0.0 0.0 0.0

In [ ]:
#np.save(filename_goes_sst, gsst)